# 분산형 공유 변수
## 브로드캐스트 변수
![broadcast-variable](../../assets/presentations/week14/broadcast-variable.png)
* 변하지 않는 값을 클로저 함수의 변수로 캡슐화하지 않고 클러스터에서 효율적으로 공유하는 방법 제공
* 모든 태스크마다 직렬화하지 않고 클러스터의 모든 머신에 캐시하는 불변성 공유 변수
* `spark.sparkContext.broadcast()`를 이용해 브로드캐스트
* `value` 메소드를 통해 브로드캐스트된 값을 참조
* 큰 크기의 데이터를 사용하는 경우 전체 태스크에서 데이터를 직렬화하는 데 부하가 커질 수 있음
* RDD 영역에서 브로드캐스트 변수 사용할 수 있으나, UDF나 Dataset에서도 사용할 수 있고 동일한 효과를 얻을 수 있음

## 어큐뮬레이터
![acuumulator](../../assets/presentations/week14/accumulator.png)
* 트랜스포메이션 내부의 다양한 값을 갱신하는 데 사용
* 내고장성을 보장하며 효율적인 방식으로 드라이버 값을 전달할 수 있음
* 결합성과 가환성을 가진 연산을 통해서만 더할 수 있는 변수임
* 병렬 처리 과정에서 효율적으로 사용 가능
* 액션 처리하는 과정에서만 단 한 번만 값을 갱신
* SparkContext를 사용해 만들거나 직접 어큐뮬레이터를 생성해 이름을 붙여 등록할 수 있음
* 이름이 지정된 어큐뮬레이터는 실행 결과를 스파크 UI에서 확인할 수 있음
* 몇 가지 기본 어큐뮬레이터를 제공하나, AccumulatorV2 클래스 상속해 사용자 정의 어큐뮬레이터를 만들 수 있음

# https://yeo0.tistory.com/entry/Spark-Spark-Application-%EC%97%90-%EB%8C%80%ED%95%9C-%EC%9D%B4%ED%95%B4-Job-Stage-Task-Transformation-Action-Lazy-Evaluation
## Application
* 스파크 위에서 돌아가는 사용자 프로그램
* 드라이버 프로그램과 executor를 지칭

## Spark Session
* 스파크 코어 기능들과 상호작용할 수 있는 진입점을 제공하는 객체
* 스파크 애플리케이션에서는 사용자가 직접 객체를 생성해야 함

## Job
* 여러 태스크로 이루어진 병렬 연산
* 스파크 애플리케이션은 하나 이상의 Job으로 변환되고, Job은 Dag (실행 계획)으로 변환됨

## Stage
* dependency를 가지는 다수의 태스크의 모임

## Task
* executor에서 실행되는 가장 작은 실행 단위
* 개별 CPU에 할당
* 개별 Partition을 가지고 작업

# https://spark.apache.org/docs/2.2.0/rdd-programming-guide.html
## 개요
* 스파크 애플리케이션은 user의 main 함수에서 실행되는 드라이버 프로그램과 클러스터에서 실행되는 다양한 병렬 연산으로 이루어짐
* 스파트의 첫 번째 주요 추상화 (abstraction, 주요 개념)는 RDD임
* 스파크의 두 번째 주요 추상화는 공유 변수임
  * 기본적으로 스파크에서 서로 다른 노드에 함수를 병렬로 실행할 경우, 각 태스크에서 사용하는 변수의 복제본을 옮지만, 각 태스크 간 혹은 태스크와 드라이버 프로그램 간 변수를 공유해야 할 필요가 있음
  * 이를 위해 스파크에서는 두 개의 공유 변수를 지원
    * 모든 노드의 메모리에 캐시되어 있는 값인 브로드캐스트 변수
    * counter와 sum과 같이 더해지는 변수인 어큐뮬레이터

## 브로드캐스트 변수
* 태스크와 함께 변수를 옮기기보다 각 노드에 읽기 전용 변수를 캐싱해둔 것
* 커뮤니케이션 비용을 줄이기 위해 효율적인 브로드캐스트 알고리즘을 이용해 브로드캐스트 변수를 분산시킴
* 분산된 shuffle 연산으로 나뉜 stage들에 의해 spark action은 실행됨
* 각 stage 안의 task에서 공통적으로 필요로 하는 데이터는 스파크에서 자동적으로 브로드캐스트함
* 위와 같은 방식으로 브로드캐스트된 데이터는 직렬화된 형태로 캐싱되어 각 태스크가 실행되기 이전에 역직렬화됨
* 여러 stage에 걸친 태스크가 같은 데이터를 이용하거나 역직렬화된 형태로 캐싱되어야 할 때 브로드캐스트 변수가 유용